## Training with a Larger Dataset - Cats and Dogs

前のラボでは、馬と人間のデータセットを使って分類器を訓練しました。素晴らしいトレーニング結果が得られたにもかかわらず、実際の画像を使って分類を行おうとすると多くのエラーが発生することがわかりました。おもに過学習が原因で、一度見たことのあるデータではネットワークはとてもよく機能しますが、見たことのないものには機能しないということが原因です。

このラボでは現実の非常に大きなデータセットを使って、過学習を回避するためにどのような影響があるかを見てみましょう。

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

数年前にKaggleのコンペで使われたデータを利用します。zipファイルをダウンロードして、tmpディレクトリに保存します。左側のペインにあるフォルダのアイコンから、ファイルを参照することができます。

In [ ]:
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset

# Note: This is a very large dataset and will take time to download

#!wget --no-check-certificate \
#    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
#    -O "/tmp/cats-and-dogs.zip"

!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


ダウンロードされた画像は、猫と犬それぞれ12,501ファイル存在します。

In [ ]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

あとで、`ImageDataGenerator` で利用しやすくするため、ディレクトリの名前をテストかトレーニング、また、画像が犬か猫であるかがわかるような名前にします。

In [ ]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

データを、指定した比率でトレーニングとテスト用として分割し、さらに犬か猫かで適切なディレクトリにコピーします。その際に選択される画像はランダムに選択されます。

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

それぞれのディレクトリに保存された画像の数をカウントします。トレーニング用としてそれぞれ、11,250ずつ。テスト用として、1,250ずつ用意されていることがわかります。（split_sizeを0.9で設定しているため）

In [ ]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

ImageDataGeneratorで正規化を行います。もともとのデータの大きさを255で割って、すべてのピクセルの値を0から1の間に収めるようにします。flow_from_directoryを利用して、指定されたデータをtrain_datagen.flow_from_directoryという形で渡すことができます。flow_from_directoryのパラメータとしては、train_dirでデータの場所を指定します。この中にはクラスの名前となるサブディレクトリを含んでいる必要があります。target_sizeには目標となる画像のサイズをいれます。入力データとしては様々な大きさの画像があることが予想されるため、大きさを統一してからニューラルネットワークに流し込む必要があります。ここで指定した大きさが入力レイヤーの大きさになりますので注意して設定する必要があります。つぎに、batch_sizeです。一度にニューラルネットワークに流し込む画像の枚数を指定します。class_modeでは、２つのクラス（犬と猫）にわける分類機を作成するので、binaryの指定となります。

In [ ]:

TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=250,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=250,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

実際のネットワークを設計します。最初の入力層には、上で説明したとおり、target_sizeとinputの大きさを一致させる必要があります。そのため、input_shapeとして150x150が指定されます。
`input_shape=(150, 150, 3)`の最後の３はRGBのカラー用に使われる次元となります。
また、最後の全結合層については１つのニューロンとなります。バイナリ型になりますが、１つのクラスには０を設定し、もう一つには１を設定することで１つのニューロンで対応が可能です。


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

構築したモデルのサマリを表示します。最後の512ニューロンをもつ全結合層については、学習する必要のあるパラメータが950万近くあるため、かなり時間がかかる可能性があります。

In [ ]:
model.summary()

訓練を開始します。1エポックあたり45秒程度かかるかもしれません。また、EXIFデータが壊れているというメッセージがでても問題ありません。これは、撮影場所等を記録するためのメタデータであり、それをpythonのモジュールが読み込もうとしてエラーとなったものです。トレーニングには影響が無いので無視していただいて結構です。

In [ ]:
# Note that this may take some time.
history = model.fit(train_generator, epochs=15, steps_per_epoch=90,
                    validation_data=validation_generator, validation_steps=6)

トレーニングとテスト精度のグラフ及び、トレーニングとテストの損失の遷移をグラフ化します。両者の精度が上昇している傾向がありますが、損失についてはテスト側の数値が上昇し、トレーニングが下がる傾向が見られます。これは過学習となってしまっている時に見られる特徴です。
（赤い線がトレーニング、青い線がテストの際の数値です）

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()


# Desired output. Charts with training and validation metrics. No crash :)

みなさんが、持ち寄った画像を読み込ませてみて、正しく犬と猫を判別するか下記のコードで試してみましょう。

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")